In [1]:
import pandas as pd

In [2]:
from bokeh.io import show
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure

In [3]:
df = pd.read_csv('logs/1533315010_track3_numenta.csv')

In [4]:
df.head()

,Iteration,Neuron,Spikes
0,-1,0,1
1,-1,1,1
2,-1,2,1
3,-1,3,1
4,-1,4,1


In [5]:
Iterations = [str(i) for i in sorted([x for x in set(df['Iteration'].tolist())])]
Neurons = [str(i) for i in sorted([x for x in set(df['Neuron'].tolist())])]

In [6]:
df ['Iteration'] = df ['Iteration'].astype(str)
df ['Neuron'] = df ['Neuron'].astype(str)

In [7]:
# this is the colormap from the original NYTimes plot
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=min(df['Spikes']), high=max(df['Spikes']))

In [8]:
TOOLS = "hover,pan,box_zoom,reset,wheel_zoom"

In [9]:
p = figure(title="Spiking Neurons over Training Iterations",
           x_range=Iterations[0::10], y_range=list(reversed(Neurons)),
           x_axis_location="above", plot_width=1920, plot_height=1080,
           tools=TOOLS, toolbar_location='below')

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1

p.rect(x="Iteration", y="Neuron", width=1, height=1,
       source=df,
       fill_color={'field': 'Spikes', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

In [10]:
show(p)